In [21]:
import pandas as pd
import spacy
import en_core_web_lg
import corenlp
from stanfordcorenlp import StanfordCoreNLP

pd.set_option('display.max_colwidth', 300)

* there is spelling errors. Probably it's an opportunity to create more examples. Just use grammarly to fix errors and add fixed examples into dataset
* Some notes on phrases
  * ambigous `who rated my page`
  * Post is negative, not reply. Probably in future we will add sentiments for replies `'Find all Marjories that replied to post negatively and downloaded exhibitor file'`
  * Poll -> Post `Find everyone who answered 'Yes' in a poll and wrote positive poll and clicked ablitech ad`
  * "Downloaded something" not realistic `Search all people with last name White who liked at least one post by Shawna Potter and downloaded something from 'sponsor' type page`
  * For now we don't have search for negative or neutral enjoyment `wasn't impressed by the event.`
  * bio -> big `Search for profiles with "bio" who liked post on biplane and replied to Cooper`
  * looks not realistic `Search attendees: ` ` x9 times `
  * looks not realistic `*3`
  * look not realistic `Find attendees:`
  * wrong ` who created a post on page with word 'easy' in text`
  * Super cool `either .. or ..` `neither .. nor ..`
  * interesting idea `name = 'Lavern'`
  * not realistic `Find the biplane people`
  * cool, but too hard; maybe need to postpone `post that can be said to be a positive one`
  * kinda cool, postpone `If there exists a post with a positive sentiment which was liked by a user who attended a single event and has a bio containing a "biplane" word -- present such users to me!`
  * not sure this is valid sentence `93 Find attendees where reply to Stacy Vaughn and phone number 7699165982 and post has "rare"`
  * ambigous `Find attendees who liked a page nine time and the page contains 'gargoyle'`
  * extra end `and and`
  * duplicate `find everyone with deviantart.com as their website`
  * what page? `saw this page`
  * too complex for users to come up `someone else's post`
  * We can't search by the note content `note that contains` `note containing the word 'concussion'` `note with the word 'concussion'` `created a note containing 'concussion'`
  * There is no such filter now `profile info contains (290) 448-8241`
  * spelling exhibotr
  * duplicate `positive positive`
  * missed closed quote `'coastal`

In [30]:
df = pd.read_csv('dataset/phrases_cleaned_1522422847595.csv')
df = df.assign(splitted_gs = None)
df.sample(10)

,id,author,created,modified,search_attrs,phrase,splitted_gs
269,59d09959-5836-494f-9176-7a11a449b2fa,Example,1522148415564,1522418681029,"[{""short"": ""by 'Acsis Inc.'"", ""type"": ""Sponsor"", ""name"": ""ad.sponsor"", ""example"": ""Attendees who clicked Apply ad with 'Wanna new iPhone?'""}, {""short"": ""with Steven Owens's mention "", ""type"": ""Attendee"", ""name"": ""post.mentions"", ""human"": ""mentioned {mention} in a post"", ""example"": ""Attendees who...","Search all attendees who mentioned Steven Owens, participated in 4 polls and converted on the 'Acsis Inc.' ad",None
146,4a98f580-3e45-4f60-9128-8195f08bd995,Michael,1522167663007,1522406848300,"[{""short"": ""5 times"", ""type"": ""int"", ""name"": ""pageView.count"", ""example"": ""Attendees who viewed sponsor page with Tesla more than 5 times""}, {""short"": ""has Vida as first name"", ""type"": ""text"", ""name"": ""profile.firstName"", ""example"": ""Attendees with first name James""}, {""short"": ""with 2 stars"", ""...",find everyone named vida,None
251,2dfad3f1-b93f-46aa-a2fa-20f390407243,Ivan,1522157665118,1522418447847,"[{""short"": ""with positive sentiment"", ""type"": ""Sentiment"", ""name"": ""post.sentiment"", ""human"": ""wrote {sentiment} post"", ""example"": ""Attendees who wrote positive post with Tesla""}, {""short"": ""4 times"", ""name"": ""events.count""}, {""short"": ""has Affordable Housing Specialist position"", ""type"": ""text""...","Look for attendees with Affordable Housing Specialist position, 4 event attendances, and positive post.",None
77,ad086d89-92c8-439f-adf4-e3dc3a62b13c,Volodymyr,1522161354671,1522226530232,"[{""short"": ""has 'free.fr' as website"", ""type"": ""text"", ""name"": ""profile.website"", ""example"": ""1""}, {""short"": ""has Greece as location"", ""type"": ""text"", ""name"": ""profile.location"", ""example"": ""Attendees with Ukraine location, *Attendees from Ukraine""}, {""short"": ""3 times"", ""name"": ""events.count""}]","Search attendees who attended event 3 times, have website set in their profile as 'free.fr', and all of them from 'Greece'",None
80,98be4665-b3db-4133-a8cf-299dfa8205ab,Kostia Rybnikov,1522249286679,1522249944131,"[{""short"": ""5 times"", ""type"": ""int"", ""name"": ""ad.count"", ""example"": ""Attendees who clicked on ad more than 2 times""}, {""short"": ""with 'exhibitor' type"", ""type"": ""PageType"", ""name"": ""notes.type"", ""example"": ""Attendees who viewed sponsor page""}, {""short"": ""0 times"", ""type"": ""int"", ""name"": ""fileDow...",Find users who clicked ad 5 times on exhibitor page without downloads,None
65,c255bb86-9172-4cc9-84a2-7e710e94106e,Andrii,1522164285913,1522218693807,"[{""short"": ""by Andy Chandler"", ""type"": ""Attendee"", ""name"": ""comment.postAuthor"", ""human"": ""wrote reply to {mention}'s post"", ""example"": ""Attenddes who wrote comment for Elon Musk post""}, {""short"": ""whose bio contains 'beyond'"", ""type"": ""text"", ""name"": ""profile.bio"", ""example"": ""1""}, {""short"": ""2...","Find attendees who replied to post written by Andy Chandler, whose bio contains 'beyond' and who viewed 2 pages.",None
200,0ea9794f-5d2f-4772-b481-49d9c691cba2,Andrew Sennik,1522172346996,1522412894449,"[{""short"": ""with 'biplane'"", ""type"": ""text"", ""name"": ""pageView.content"", ""example"": ""Attendees who viewed page with SpaceX""}, {""short"": ""3 times"", ""type"": ""int"", ""name"": ""notes.count"", ""example"": ""Attendees who viewed sponsor page with Tesla more than 5 times""}, {""short"": ""7 times"", ""type"": ""int...","Find all attendees that made 7 file downloads, visited a page that contains 'biplane', and created 3 notes.",None
188,0ea9794f-5d2f-4772-b481-49d9c691cba2,Andrew Sennik,1522172346996,1522412894449,"[{""short"": ""with 'speaker' type"", ""type"": ""PageType"", ""name"": ""pageView.type"", ""example"": ""Attendees who viewed sponsor page""}, {""short"": ""with negative sentiment"", ""type"": ""Sentiment"", ""name"": ""post.sen

In [25]:
snlp = StanfordCoreNLP(path_or_host='http://localhost', port=9000)

In [27]:
print(snlp.parse("Find people who wrote posts mentioning Stacy Vaughn and liked any post 9 times and clicked on ad with 'blackout' in it"))

(ROOT
  (S
    (VP
      (VP (VB Find)
        (NP
          (NP (NNS people))
          (SBAR
            (WHNP (WP who))
            (S
              (VP
                (VP (VBD wrote)
                  (NP (NNS posts))
                  (S
                    (VP (VBG mentioning)
                      (NP (NNP Stacy) (NNP Vaughn)))))
                (CC and)
                (VP (VBD liked)
                  (NP (DT any) (NN post))
                  (NP-TMP (CD 9) (NNS times))))))))
      (CC and)
      (VP (VBD clicked)
        (PP (IN on)
          (NP (NN ad)))
        (PP (IN with)
          (NP (`` `)
            (NP (NN blackout))
            ('' ')
            (PP (IN in)
              (NP (PRP it)))))))))


In [29]:
print(snlp.parse("Find attendees who liked a post by Philip Stephens, left a note on a page 3 times, and downloaded a file named workshop.doc"))

(ROOT
  (S
    (VP (VB Find)
      (NP
        (NP (NNS attendees))
        (SBAR
          (WHNP (WP who))
          (S
            (VP
              (VP (VBD liked)
                (NP (DT a) (NN post))
                (PP (IN by)
                  (NP (NNP Philip) (NNP Stephens))))
              (, ,)
              (VP (VBD left)
                (NP (DT a) (NN note))
                (PP (IN on)
                  (NP
                    (NP (DT a) (NN page))
                    (NP-TMP (CD 3) (NNS times)))))
              (, ,)
              (CC and)
              (VP (VBD downloaded)
                (NP
                  (NP (DT a) (NN file))
                  (VP (VBN named)
                    (NP (NN workshop.doc))))))))))))


In [31]:
print(snlp.parse("Look for attendees with Affordable Housing Specialist position, 4 event attendances, and positive post."))

(ROOT
  (S
    (VP (VB Look)
      (PP (IN for)
        (NP
          (NP (NNS attendees))
          (PP (IN with)
            (NP
              (NP (JJ Affordable) (NN Housing) (NN Specialist) (NN position))
              (, ,)
              (NP (CD 4) (NN event) (NNS attendances))
              (, ,)
              (CC and)
              (NP (JJ positive) (NN post)))))))
    (. .)))
